# Full Business Solution

#### My solution: Email Reply Assistant for small businesses

This is an example of a tool that's been able to respond to customer emails in a helpful way, by analyzing the customer's message and extracting key information from it, and then using that information to craft a response.

In [102]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [103]:
load_dotenv(override=True)

openai_key = os.getenv('OPENAI_API_KEY')

if not openai_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not openai_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


### Processing inputs

First, input the customer message.

In [104]:
customer_message = str(input("Customer message: "))

Making the corresponding prompts for the LLM

In [105]:
system_prompt = """
You are a very helpful assistant that collects information about customer emails. You should analize those messages and extract the following information:

1) The customer's name (if possible)
2) Intent of the message (what is the customer asking for?)
3) Urgency
4) Sentiment of the message (is the customer happy, sad, angry, etc?)
5) Any other relevant information that can be extracted from the message
6) Missing info needed (list)
7) Recommended next steps (list)

Please return all of this information in a JSON format, with the following structure:

{
    "customer_name": "",
    "language": "",
    "intent": "",
    "urgency": "",
    "sentiment": "",
    "other_info": "",
    "missing_info": [],
    "recommended_next_steps": []
}
"""

In [106]:
def displayJsonResponse(message) -> dict:
    user_prompt = f"""
    Customer message: {message}
    """
    openai = OpenAI()
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )

    response_content = json.loads(response.choices[0].message.content)
    return response_content


### Sending responses according to JSON info

I will use another LLM, let's try with Gemini

In [107]:
gemini_api = os.getenv('GOOGLE_API_KEY')
gemini_base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

if not gemini_api:
    print("No Gemini API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not gemini_api.startswith("AIz"):
    print("A Gemini API key was found, but it doesn't start with AIza; please check you're using the right key - see troubleshooting notebook")
else:
    print("Gemini API key found and looks good so far!")

Gemini API key found and looks good so far!


Upload system prompt and user prompt for the new LLM call

In [108]:
system_prompt_gemini = """
You are a customer-support email assistant.
You will receive a JSON analysis of the customer's message with fields like:
intent, urgency, sentiment, missing_info, recommended_next_steps.

Requirements:
- Plain text only (no markdown, no code fences, no surrounding quotes).
- Keep natural line breaks.
- Always write in first-person plural: "we", "our", "us".
- Never use first-person singular: "I", "me", "my".
- Professional, warm, concise tone.
- Reply in the same language as the customer.
- For return/devolution requests, ask politely for required details:
  order number, purchase date, issue description, and return address.

At the end of the email, include a text that says:
"Kind regards,"
"Client Support Team"

Possible intents:
Greeting, Product Inquiry, Pricing, Order Status, Shipping/Delivery,
Return/Refund/Devolution, Exchange, Complaint, Technical Support,
Billing/Payment, Cancellation, Warranty, Escalation, Other.

Intent handling:
- Greeting: greet and ask how you can help.
- Product Inquiry/Pricing: provide concise info and ask needed details.
- Order Status/Shipping: ask for order number if missing.
- Return/Refund/Devolution: ask for order number, product, reason, and full return address.
- Complaint: acknowledge issue, apologize, ask key details, propose next step.
- Technical Support: ask for error details, device/app version, and steps tried.
- Billing/Payment: ask invoice/order ID and payment date/method.
- Cancellation: confirm request and ask order/subscription ID.
- Other: ask a clarifying question.

If key data is missing, ask only for the minimum required details.
"""


In [109]:
def sendEmailResponse(msg) -> str:
    analysis = displayJsonResponse(msg)
    user_prompt_gemini = f"""
    Here is the JSON analysis of the customer's message:
    {json.dumps(analysis, ensure_ascii=False, indent=2)}
    """

    gemini = OpenAI(base_url=gemini_base_url, api_key=gemini_api)

    response_gemini = gemini.chat.completions.create(
        model="gemini-2.5-flash-lite",
        messages=[
            {"role": "system", "content": system_prompt_gemini},
            {"role": "user", "content": user_prompt_gemini}
        ],
        temperature=0.2
    )
    return response_gemini.choices[0].message.content.strip()


In [110]:
email_text = sendEmailResponse(customer_message)
print(email_text)

Thank you for your kind words! We are delighted to hear that you are satisfied with our services and look forward to continuing our business relationship.

Please do not hesitate to reach out if you need any assistance in the future.

Kind regards,
Client Support Team
